# Introduction to Py6S: A Radiative Transfer Model using Python

## Py6S: A Python interface to 6S

<img src="6s_diagram_only_diagram.png", style="width: 50%", align="right">The **S**econd **S**imulation of the **S**atellite **S**ignal in the **S**olar **S**pectrum (**6S**) is an atmospheric Radiative Transfer Model that makes possible to simulate the electromagnetic radiation through a planetary atmosphere. Py6S provides a simple interface through the Python programming language allowing you to run many 6S simulations using a simple Python syntax, rather than dealing with 6S input and output files. As well as generally making it easier to use 6S, Py6S adds a number of new features including:

   * The ability to run many simulations easily and quickly, with no manual editing of input files
   * The ability to run for many wavelengths and/or angles and easily plot the results
   * The ability to import real-world data to parameterise 6S, such as radiosonde measurements, AERONET sun photometer measurements and ground reflectance spectra from spectral libraries

Py6S is copyright Robin Wilson and code is freely available at: 
    * https://github.com/robintw/py6s



Documentation including how to install the software and an extensive "quick start guide" are also available at: https://py6s.readthedocs.io/en/latest/ During this training we will use some of those resources combined with new examples and radiative transfer model theory to quickly give an overview of Py6S and its potential.

## Introduction to Jupyter

During this practical exercise you will be using a Jupyter Notebook (http://jupyter.org/). The Jupyter Notebook is an interactive computational environment, in which you can combine code execution, rich text, mathematics, plots and media.  You will also be able to run Python code by pressing the "play" button at the top so even if you are not familiar with Python you should be able to work your way through this worksheet. If you are already familiar with Jupyter Notebooks feel free to skip these few lines, if not, please try running  the following line of code using the "Run" button at the top of the notebook.

In [ ]:
print("Hello World!")

As well as running existing code, you will also be able to edit the code and write your own pieces of code. For example, click on the following box and change the value of `example_str` to `"Hi!"`. Press the "play" button at the top and run the code.

In [ ]:
example_str = "You can edit this sentence and press play!" 
#This comment is to remind you that quotes are necessary
print(example_str)

If you followed the instructions, the above output should be "Hi!". You can write much more complicated code but that simple (and classic) example should be enough to get you to the issue at hand: **Py6S**

## Getting started with Py6S

To take advantage of the Jupyter Notebook we will start with a simple Py6S example. The following code will run the 6S model across the Visible-NIR wavelength range using the default values. Later on we will introduce different parameters for the model in detail. Please read the following code then run (using the "Run" button). It might take a couple of minutes to run through, once it has finished it will display a plot.

In [ ]:
%matplotlib inline
#This is specific for Jupyter to show the plots in this same window

#Now we start by importing the Py6S module
import Py6S
# Create a SixS object called s (used as the standard name by convention)
s = Py6S.SixS()
# Will run the 6S simulation defined by this SixS object across the whole VNIR range
wavelengths, results = Py6S.SixSHelpers.Wavelengths.run_vnir(s, output_name="pixel_radiance", verbose=False)
#Verbose=False will allow us to not print in the screen the entire array plotted in the screen
#Will plot these results, with the y axis label set to "Pixel Radiance"
Py6S.SixSHelpers.Wavelengths.plot_wavelengths(wavelengths, results, "Pixel Radiance")
#This will take a few seconds to plot the entire numpy array
print("All done!")


As you have noticed, Py6S has built in helper function for plotting data which are useful for quickly checking outputs. In this training we will be using [matplotlib](https://matplotlib.org/) directly, as this allows more control over plotting parameters.

The sections of code in a Jupyter notebook are executed sequentially so variables defined in an earlier section of code are available within any sectons of code defined after. For example, when you executed:

In [ ]:
import Py6S

The imported module will be already in memory and will not be needed to be imported again. During the begining of this training I will be writing the entire code and importing modules into each block of code for clarity. This should help you getting used to the coding styles if you are not familiar with Python but later on I will only write the specific lines needed. You can always come back to this training to finish it at a later date. If you want to to use Py6S on your own you will want to be able to copy paste the code used in this training, apply some minor changes and have it run sucessfully. 

## Parameters for any Radiative Transfer Model

Before explainning in detail the different parameters available on Py6S, it is important to understand the main parameters that regulate any solar radiation transfer in the atmosphere. We can simplify the model and summarize those main variables as:

* Water vapour content
* Aerosol composition and thickness
* Geometry: Position of illumination (the Sun) and sensor relative to the target
* Ground reflectance characteristics


### Atmospheric Profile

In the solar spectrum the atmospheric gaseous absorptions are principally due to: oxygen, ozone, water vapour, carbon dioxide, methane and nitrous oxide.

<img src="water_vapor_columns.png", style="width: 30%", align="right"> Typical water contents, in grams per square centimetre, for different zones are shown in the table to the right.

You can add your own water vapour value if you have a measure of it or you can use it or you can download it if available for some web services such as AERONET. You can also calculate the water content by the latitude and date using a built in Py6S method but this will be approximate and will depend on how close your site is to the sea or the weather for that specific day.

It is also possible to estimate water vapour from an image spectra, by using water absorption bands. This is the method used by some atmospheric correction packages such as ATCOR.

Have a look at what different default values and methods there are for providing an atmospheric profile in Py6S. If you have been running the previous code, the Py6S module is already imported and you will be able to display the available default options by clicking into the next code and pressing *TAB*. Give it a go:

In [ ]:
Py6S.AtmosProfile.

Please note that those are predefined methods and they do not return water vapour. Each method stores an enumeration for the pre-specified atmospheric model types. If you want to use a specific value of water vapour you can use other different class methods such as: `Py6S.AtmosProfile.UserWaterAndOzone(water, ozone)` 

For example:

In [ ]:
s.atmos_profile = Py6S.AtmosProfile.PredefinedType(Py6S.AtmosProfile.Tropical)

### Geometry

The geometry should include solar angles and altitude of target and sensor. 

<img src="solar_angles_small.png", style="width: 40%", align="left">  Using the Py6S module, we can compute the position of the sun by using the latitude and longitude of the target as well as time and day of the year. For the sensor we still need the angles:

* View azimuth: Angle in the ground between North and the sensor view direction
* View zenith angle: Angle between the nadir and the sensor view direction

For example, for an observer directly under the flight path of the plane, the azimuth angle will be the same as the direction of the plane (heading) and the view zenith angle will be 0.


In the next practical example we will examine how different viewing angles can affect the spectra. For a given position and time, test different viewing angles of 0° (nadir) and 60°. Run the following code to see how the spectra changes. It will take a couple of minutes to run and will generate a plot.

In [ ]:
%matplotlib inline
#This is specific for Jupyter to show the plots in this same window

import Py6S
import numpy
from matplotlib import pyplot as plt

#Will define a location by its lat and lon
lat = 50.366110
lon = -4.148037
#Now we need the date and time in a string with the following format:
flight_time = '13/02/2017 09:00:00'
#Flight altitude
plane_altitude = 1 #Altitude in km
#Will choose the spectral range between 0.38 and 2.5 micrometres 
wavelengths = numpy.arange(0.38, 2.5, 0.025)
#Please note that Py6S needs the input to be in micrometres


#We can create the Py6S.SixS() class
s = Py6S.SixS()
#We will define our own geometry
s.geometry = Py6S.Geometry.User()
#With our own plane altitude
s.altitudes.set_sensor_custom_altitude(plane_altitude)

#Let's make this very simple and have one class with viewing angle 0
s0 = s
#We have just copied all definitions of class 's' to a new class 's0'
view_zenith = 0
#The azimuth angle will be 0
view_azimuth = 0
#Now we add the location and angles to our geometry s0
s0.geometry.from_time_and_location(lat, lon, flight_time, view_zenith, view_azimuth)
#Now we can compute the Py6S model for our class s0 with the geometry included; this might take some time
py6s_wv, py6s_output_0 = Py6S.SixSHelpers.Wavelengths.run_wavelengths(s0, wavelengths, 
                                                                  output_name='apparent_radiance',
                                                                  verbose=False)
#Now we can plot it: please note that the output has been named as "py6s_output_0"
plt.plot(py6s_wv*1000, py6s_output_0*100, label="Viewing zenith angle = {}".format(view_zenith))
#We have changed the units to match the default NERC-ARF Fenix sensor (so can compare easily)
#We have transformed the wavelengths from micrometres to nanometres and
#the at-sensor radiance to nW cm$^{-2}$ sr$^{-1}$ nm$^{-1}$'
#We will use the same notation from now on


#Now let's do the same with viewing_angle=60 -> will name the new class s60
s60 = s
#We have just copied all definitions of class 's' to a new class 's60'
view_zenith = 60
#In this case, the azimuth angle has been changed to be 60 instead of 0
#The azimuth angle will still be 0 
view_azimuth = 0
#We add the new parameters to the geometry and run model
s60.geometry.from_time_and_location(lat, lon, flight_time, view_zenith, view_azimuth)
py6s_wv, py6s_output_60 = Py6S.SixSHelpers.Wavelengths.run_wavelengths(s60, wavelengths, 
                                                                  output_name='apparent_radiance',
                                                                  verbose=False)
#And we plot it as well, this time the output has been named as "py6s_output_60"
plt.plot(py6s_wv*1000, py6s_output_60*100, label="Viewing zenith angle = {}".format(view_zenith))

plt.xlabel('Wavelength (nm)')
plt.ylabel('At-sensor radiance nW cm$^{-2}$ sr$^{-1}$ nm$^{-1}$')
plt.legend()
plt.show()

The latest code makes a perfect example about how you can use Py6S for your data. You can install the software (or keep using this virtual machine) and just need to edit the input to your geometry to match your needs. You will have an usable script that will allow you to compare the model with your recorded data. As part of the delivery checks, the NERC-ARF-DAN have an automated script that looks for vegetation pixels in any hyperspectral data and compares it with the spectra simulated by 6S at the plane altitude. This easily and quickly allows to detect possible problems with the spectra if the sensor developed a fault. 

Now, how will the spectra change if you use the same parameters as before but have a constant viewing angle and variable azimuth? Try it yourself, copy the above code in the next box and edit it before running. Use the parameters:
    * s0  -> Azimuth=0°, zenith=60°
    * s60 -> Azimuth=60°, zenith=60°

Take a moment to see how the spectra varies. Now what if you set both zenith angles to 0 and compare azimuth angles from 0° and 60°, how will the spectra vary? Why? Take a moment to think about this and edit the latest code to meet the new criteria and test it.

In the case of aerosol composition, if you know the atmosphere composition you can specify the percentages to have a more accurate model. You can specify contributions of each aerosol using the class method AeroProfile.User with the keywords:

    dust – The proportion of dust-like aerosols
    water – The proportion of water-like aerosols
    oceanic – The proportion of oceanic aerosols
    soot – The proportion of soot-like aerosols

Please note that the sum of proportions must be one. Here you have a couple of examples:

In [ ]:
s.aeroprofile = Py6S.AeroProfile.User(dust=0.3, oceanic=0.7)
#or:
s.aeroprofile = Py6S.AeroProfile.User(soot = 0.1, water = 0.3, oceanic = 0.05, dust = 0.55)
#Note that for both lines, the sum of percentages is 1. Otherwise Py6S will show an error

### Aerosol Optical Thickness

Another important parameter is the Aerosol Optical Thickness (AOT) defined at 550nm, which is related to the visibility. It can be measured (e.g., using a sun photometer) or imported from a service such as [AERONET](https://aeronet.gsfc.nasa.gov/). There are specific functions developed within `Py6S.SixSHelpers` that allow you to define the aerosol profile by importing AERONET measurements or using the  British Antarctic Survey radiosonde format but in the simplest case scenario, you can enter the values manually.

In the next example we will go one step beyond and will use a loop over different AOT values to see how the spectra varies. From the following code, I will not repeat earlier code and import modules / define variables again, as they should have been loaded into memory. Just remember that if you come back to this workseet, you will have to run again the code shown before that imported modules and created the Py6S classes).

Please read the code carefully and run it. Again it will take a couple of minutes to run and produce a plot.

In [ ]:
plane_altitude = 0.2 #Altitude in km
#Let's use this smaller altitude to see variations more clearly
s.altitudes.set_sensor_custom_altitude(plane_altitude)
#With the last line we have included that new altitude into account

#Will do a loop over optical thickness varying from 0 to 1 with intervals of 0.2
for aot_val in numpy.arange(0, 1, 0.2):
    #Everything that is at this level of indentation is running in the loop
    print("Will run 6S model for AOT = {:.2f}".format(aot_val))
    #Have printed an informative message and will define the aerosol optical thickness
    s.aot550 = aot_val
    #Will run the model
    py6s_wv, py6s_output = Py6S.SixSHelpers.Wavelengths.run_wavelengths(s, wavelengths, 
                                                                  output_name='apparent_radiance',
                                                                  verbose=False)
    #And plot
    plt.plot(py6s_wv*1000, py6s_output*100, label="AOT={:.2f}".format(aot_val))
    #End of loop
    
plt.xlabel('Wavelength (nm)')
plt.ylabel('At-sensor radiance nW cm$^{-2}$ sr$^{-1}$ nm$^{-1}$')
plt.legend()
plt.show()

In this simple latest example you have seen how easily you can compute different models even if you are not familiar with programming languages. Would you be able to do a similar loop over plane altitudes from 1km to 20 kms by increasing the plane altitude 5 kms on each iteration? Try yourself by copying and editing the latest code in the next box!

### Ground Reflectance Characteristics

The properties of the target and reflectance properties from different angles effects how much radiation is received by the sensor. When the target behaves like a mirror we have specular reflection,  where each ray is reflected at the same angle as the incident ray. If the surface is not regular, we have diffuse reflection. The diffuse reflection is the type of reflection we can observe from water bodies.


<img src="difuse_vs_specular.png", style="width: 70%", align="center">

However, natural surfaces do reflect light in all directions. An ideal reflective surface that reflects  light in all directions is called a Lambertian surface. <img src="lambertian_surface.png", style="width: 40%", align="center">

Please note that the Lambertian surface reflects light in all directions homogeneously. Natural surfaces usually exhibit a non homogeneous anisotropic reflectance characteristics, which describe the dependency of the reflectance spectra on the illumination and viewing geometry.  This is defined by the Bidirectional Reflectance Distribution Function (BRDF) and lambertian reflectances are usually used as a first approximation of the model.

Py6S has a  number of different ground reflectances that include homogeneous and heterogenous reflectances. Without entering into much detail about the different models available (Lambertian homogeneous or heterogeneous, homogeneous Hapke...); it is worth pointing out that Py6S also contains some predefined ground reflectances for different types of surfaces such as green vegetation, water... With that in mind, if you would like for example to use the homogenous lambertian model for a green vegetation area you will be using:

In [ ]:
s.ground_reflectance = Py6S.GroundReflectance.HomogeneousLambertian(Py6S.GroundReflectance.GreenVegetation)

Once again, you can have a look at all types of predefined methods available by pressing *TAB* for each class. For example, click in the next box and press *TAB* to see all methods available:

In [ ]:
Py6S.GroundReflectance.

As you have seen there are plenty of options! You have now had a quick overview about the main parameters that affect Radiative Transfer Models, how they modulate the spectra as well as how to specify within the Py6S model. You are ready now to have a look at all different parameters that Py6S (as part of the SixS class methods) can handle:

<div class="wy-table-responsive"><table class="docutils" border="1">
<colgroup>
<col width="15%">
<col width="44%">
<col width="41%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">SixS Parameter</th>
<th class="head">Description</th>
<th class="head">Possible values</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><code class="docutils literal"><span class="pre">atmos_profile</span></code></td>
<td>Atmospheric profile (pressure, water vapour, ozone etc)</td>
<td>Any outputs from <a class="reference internal" title="Py6S.AtmosProfile"><code class="xref py py-class docutils literal"><span class="pre">AtmosProfile</span></code></a></td>
</tr>
<tr class="row-odd"><td><code class="docutils literal"><span class="pre">aero_profile</span></code></td>
<td>Aerosol profile (types, distributions etc)</td>
<td>Any outputs from <a class="reference internal" title="Py6S.AeroProfile"><code class="xref py py-class docutils literal"><span class="pre">AeroProfile</span></code></a></td>
</tr>
<tr class="row-even"><td><code class="docutils literal"><span class="pre">ground_reflectance</span></code></td>
<td>Ground reflectance (Homogeneity, BRDF etc.)</td>
<td>Any outputs from <a class="reference internal"  title="Py6S.GroundReflectance"><code class="xref py py-class docutils literal"><span class="pre">GroundReflectance</span></code></a></td>
</tr>
<tr class="row-odd"><td><code class="docutils literal"><span class="pre">geometry</span></code></td>
<td>Viewing/Illumination geometry (manual or satellite-specific)</td>
<td>A <code class="docutils literal"><span class="pre">Geometry*</span></code> class, for example <a class="reference internal"  title="Py6S.Geometry.User"><code class="xref py py-class docutils literal"><span class="pre">Geometry.User</span></code></a></td>
</tr>
<tr class="row-even"><td><code class="docutils literal"><span class="pre">aot550</span></code></td>
<td>Aerosol Optical Thickness at 550nm</td>
<td>Floating point number</td>
</tr>
<tr class="row-odd"><td><code class="docutils literal"><span class="pre">visibility</span></code></td>
<td>Visibility in km</td>
<td>Floating point number</td>
</tr>
<tr class="row-even"><td><code class="docutils literal"><span class="pre">altitude</span></code></td>
<td>Altitudes of the sensor and target</td>
<td>An instance of the <code class="docutils literal"><span class="pre">Altitudes</span></code> class</td>
</tr>
<tr class="row-odd"><td><code class="docutils literal"><span class="pre">atmos_corr</span></code></td>
<td>Atmospheric correction settings (yes/no, reflectances)</td>
<td>Any outputs from <a class="reference internal" title="Py6S.AtmosCorr"><code class="xref py py-class docutils literal"><span class="pre">AtmosCorr</span></code></a></td>
</tr>
</tbody>
</table></div>

In the latest examples we have not specified any atmospheric correction but as you can see in the table a few models are available in the 6S model. This training is just a quick introduction to give you a feeling about how easily you can use the 6S model for your research. For more information about all different options and setup for the parameters as well as how to use atmospheric correction within the model; please visit the  Py6S documentation page.

## Extended Py6S Options Helpers methods, accessing the outputs and use of field spectra

You might have noticed that we have been always running the model by using one of the SixSHelpers methods built in Py6S, more precisely by running:

    Py6S.SixSHelpers.Wavelengths.run_wavelengths(s, wavelengths,output_name)

While you can choose to run the model and access the outputs once finished (as we will later show), the SixSHelpers module provides a number of helper functions that makes it easier to run the model for specific data sources as well as producing the wavelengths and BRDF plots from 6S runs.

The main types of SixSHelpers include:
   - Py6S.SixSHelpers.Wavelengths
   - Py6S.SixSHelpers.Angles
   - Py6S.SixSHelpers.Radiosonde
   - Py6S.SixSHelpers.Aeronet
   - Py6S.SixSHelpers.Spectra

We will not go over each method but you can, for example, run the model using the wavelengths of the MERIS satellite by running Py6S.SixSHelpers.Wavelengths.run_meris or easily plot the spectra with the already available method SixSHelpers.Wavelengths.plot_wavelengths

Use the next box to have a look to the available SixSHelpers.Wavelengths. methods by clicking right afer the dot and pressing TAB.

In [ ]:
help(Py6S.SixSHelpers.Wavelengths. )
#If you select a method and run this code, it will print the documentation written in the code
#It might be very technical and large...

On the other hand, if you do not use one of the helper methods, you can run the model and access the outputs one by one with the tab. Have a look at the following example:

In [ ]:
#We import the module
import Py6S
#Create a default SixS class
s = Py6S.SixS()
#Will run the model for the default
s.run()
#All done, we can access the outputs stored in the class:
print(s.outputs.transmittance_water)

Once again, play a bit with the outputs available using the following box.

### Importing Spectra

If you are using some ground spectra, you can also use it to model the ground reflectance. The helper methods include a function to import a spectral library from the ASTER Spectral Library and another method to use the USGS Spectral library. However, if you recorded your own ground spectra you can use it too. That is the magic of Python!

To finalise this short practical session, in the next example you will see how to import your own spectra by using the PySpectra module. You can create an spectra object by specifying the path to the file and the scale (percentage) of the reflectance measured. Read the following code and run it to load the ground spectra within the file (also downloaded with this jupyter notebook):

In [ ]:
#We need to import the new library
import PySpectra
#we will also use later numpy as well so let's import it now
import numpy

#Now you can create a spectra object
spectra_object = PySpectra.extract_spectra_from_file("ground_measure.txt",
                                                    reflectance_scale=float(100))
#TIP: If the file is in another directory, you will neeed to specify its path
#Note that the relfectance_scale depends on your data (usually percentages)

If you have been able to create the spectra object, it will have stored the attribute wavelengths and values as:
     spectra_object.wavelengths
     spectra_object.values

Now you can use it to compute the ground reflectance. In order to do so, you have to extract the wavelengths and reflectances to compute the ground reflectance matrix. This code might get a bit more complicated than before so even if you are not used to coding, please stay with me; we are close to the end! To compute the ground reflectance matrix you can do:

In [ ]:
#First of all remove values outside reflectance limits
spectra_object.wavelengths = spectra_object.wavelengths[spectra_object.values < 1.0]
spectra_object.values = spectra_object.values[spectra_object.values < 1.0]

#Now we want to use the wavelengths and values for the matrix
val = spectra_object.values
wl = spectra_object.wavelengths
#TIP: This is really important, Py6S needs micrometers! 
if spectra_object.wavelength_units == 'nm':
    #If you are using the provided data it would be nanometres
    #Need to convert from nanometers to micrometres for py6s
    wl = wl*0.001
else:
    #For this training it has to be so will throw an error but you can change this if not
    raise Exception("Unknown wavelength units: {}".format(spectra_object.wavelength_units))
#Now we can compute the array GR = Ground reflectance
GR = numpy.array([wl,val])
#But note it has to be the transponse of the matrix!
GR = GR.transpose()
#Finally ready!

Once the final **GR** array has been created, you can just enter it into the Py6S model. Assuming that in this case we can use the homogeneous Lambertian approximation, we can simply enter that into the SixS class already created by using the ground_reflectance parameter:

In [ ]:
s.ground_reflectance = Py6S.GroundReflectance.HomogeneousLambertian(GR)

And we are finally in the position to run the model using our ground reflectance. As a final exercise, would you be able to run the model now in the next box and plot the results? Would you be able to use what you have learned to compare in a single plot the results of using our own ground reflectance and one of the predefined types such as the Py6S.GroundReflectance.ClearWater? 

Dare yourself to try it out in the next box

We hope that you have enjoyed this short hands-on training. Hopefuly you have realised how easily the 6S model can be run using the default types included in Py6S and how you can get more accurate results with a bit of time by using other methods. If you think this might be beneficial for your research it definetly pays off! Once again, Py6S is freely available and a more extensive guide is available here:
https://py6s.readthedocs.io/en/latest/ 